In [1]:
with open('input') as f:
    intcode = [int(n) for n in f.read().split(',')]

In [2]:
def process_intcode(intcode, input_values=None):
    intcode = {i: instruction for i, instruction in enumerate(intcode)}
    if input_values is not None:
        driving_mode = True
        intcode[0] = 2
        input_values = iter(input_values)
    else:
        driving_mode = False
    relative_base = 0
    i = 0
    grid = set()
    start_pos = None
    start_direction = None
    x, y = (0, 0)
    while True:
        initial = str(intcode.get(i, 0)).rjust(5, '0')
        a, b, c, *opcode = initial
        opcode = int(''.join(opcode))
        assert opcode in (1, 2, 3, 4, 5, 6, 7, 8, 9, 99), opcode
        
        if opcode == 99:
            if driving_mode:
                return output_value
            return grid, start_pos, start_direction

        if c == '0':
            p1 = intcode.get(i + 1, 0)
        elif c == '1':
            p1 = i + 1 
        elif c == '2':
            p1 = intcode.get(i + 1, 0) + relative_base

        if b == '0':
            p2 = intcode.get(i + 2, 0)
        elif b == '1':
            p2 = i + 2
        elif b == '2':
            p2 = intcode.get(i + 2, 0) + relative_base

        if a == '0':
            p3 = intcode.get(i + 3, 0)
        elif a == '1':
            p3 = i + 3
        elif a == '2':
            p3 = intcode.get(i + 3, 0) + relative_base

        if opcode == 1:
            intcode[p3] = intcode.get(p1, 0) + intcode.get(p2, 0)
            i += 4
        elif opcode == 2:
            intcode[p3] = intcode.get(p1, 0) * intcode.get(p2, 0)
            i += 4
        elif opcode == 3:
            input_value = next(input_values)
            intcode[p1] = input_value
            i += 2
        elif opcode == 4:
            if driving_mode:
                output_value = intcode[p1]
            else:
                output_value = chr(intcode.get(p1, 0))
                if output_value == '\n':
                    y += 1
                    x = 0
                else:
                    if output_value == '#':
                        grid.add((x, y))
                    elif output_value == '^':
                        start_pos = (x, y)
                        start_direction = output_value
                        grid.add((x, y))
                    x += 1
            i += 2
        elif opcode == 5:
            if intcode.get(p1, 0) != 0:
                i = intcode.get(p2, 0)
            else:
                i += 3
        elif opcode == 6:
            if intcode.get(p1, 0) == 0:
                i = intcode.get(p2, 0)
            else:
                i += 3
        elif opcode == 7:
            intcode[p3] = 1 if intcode.get(p1, 0) < intcode.get(p2, 0) else 0
            i += 4
        elif opcode == 8:
            intcode[p3] = 1 if intcode.get(p1, 0) == intcode.get(p2, 0) else 0
            i += 4
        elif opcode == 9:
            relative_base += intcode.get(p1, 0)
            i += 2

In [3]:
def display_grid(grid, start_pos, start_direction):
    max_x = max(p[0] for p in grid)
    max_y = max(p[1] for p in grid)

    print('   ' + ''.join(str(x)[-1] for x in range(max_x + 1)))
    for y in range(max_y + 1):
        print(f"{y:2.0f}", end=' ')
        for x in range(max_x + 1):            
            if (x, y) == start_pos:
                char = start_direction
            elif is_intersection(x, y, grid):
                char = 'O'
            elif (x, y) in grid:
                char = '#'
            else:
                char = '.'
            print(char, end='')
        print()

In [4]:
def get_neighbours(x, y):
    return {(x, y - 1), (x, y + 1), (x - 1, y), (x + 1, y)}

In [5]:
def is_intersection(x, y, grid):
    return (x, y) in grid and all((dx, dy) in grid for dx, dy in get_neighbours(x, y))

In [6]:
def get_intersections(grid):
    return {
        (x, y)
        for (x, y) in grid
        if is_intersection(x, y, grid)
    }

In [7]:
grid, start_pos, start_direction = process_intcode(intcode)
display_grid(grid, start_pos, start_direction)

   0123456789012345678901234567890123456789012345678
 0 ........................................#########
 1 ........................................#.......#
 2 ........................................#.......#
 3 ........................................#.......#
 4 ......................#########.........#.......#
 5 ......................#.......#.........#.......#
 6 ......................#.......#.........#.......#
 7 ......................#.......#.........#.......#
 8 ......................#.......#.........#.......#
 9 ......................#.......#.........#.......#
10 ..........###########.#.......#.......##O########
11 ..........#.........#.#.......#.......#.#........
12 ..........#.........#.#.......########O##........
13 ..........#.........#.#...............#..........
14 ..........#...######O##...............#..........
15 ..........#...#.....#.................#..........
16 ..........#...#.....#.................#..........
17 ..........#...#.....#.................#....

In [8]:
intersections = get_intersections(grid)
print("Part 1:")
print(sum([a * b for a, b in intersections]))

Part 1:
6212


In [9]:
def look_forward(pos, direction):
    x, y = pos
    return {
        '^': (x, y - 1),
        'v': (x, y + 1),
        '>': (x + 1, y),
        '<': (x - 1, y),
    }[direction]

In [10]:
def turn(grid, pos, direction):
    x, y = pos
    left_pos, right_pos = {
        '^': ((x - 1, y), (x + 1, y)),
        'v': ((x + 1, y), (x - 1, y)),
        '>': ((x, y - 1), (x, y + 1)),
        '<': ((x, y + 1), (x, y - 1)),
    }[direction]
    if left_pos in grid:
        new_direction = {
            '^': '<',
            'v': '>',
            '>': '^',
            '<': 'v',
        }[direction]
        return (left_pos, 'L', new_direction)
    if right_pos in grid:
        new_direction = {
            '^': '>',
            'v': '<',
            '>': 'v',
            '<': '^',
        }[direction]
        return (right_pos, 'R', new_direction)
    raise ValueError()

In [11]:
def reduce_steps(steps):
    c = 0
    last_step = None
    for step in steps:
        if step == 'F':
            c += 1
        else:
            if last_step == 'F':
                yield c
                c = 0
            yield step
        last_step = step
    yield c

In [12]:
def navigate(grid, start_pos, start_direction):
    pos = start_pos
    direction = start_direction
    steps = []
    visited = {pos}
    while len(visited) < len(grid):
        x, y = pos
        next_pos = look_forward(pos, direction)
        if next_pos in grid:
            pos = next_pos
        else:
            pos, left_right, direction = turn(grid, pos, direction)
            steps.append(left_right)
        steps.append('F')
        visited.add(pos)
    steps = list(reduce_steps(steps))
    assert verify_route(grid, start_pos, start_direction, steps)
    return steps

In [13]:
def verify_route(grid, pos, direction, steps):
    visited = {pos}
    for step in steps:
        if type(step) is int:
            for i in range(step):
                x, y = pos
                pos = {
                    '^': (x, y - 1),
                    'v': (x, y + 1),
                    '>': (x + 1, y),
                    '<': (x - 1, y),
                }[direction]
                visited.add(pos)
        else:
            direction = {
                ('^', 'L'): '<',
                ('^', 'R'): '>',
                ('v', 'L'): '>',
                ('v', 'R'): '<',
                ('>', 'L'): '^',
                ('>', 'R'): 'v',
                ('<', 'L'): 'v',
                ('<', 'R'): '^',
            }[(direction, step)]
    return visited == grid

In [14]:
steps = navigate(grid, start_pos, start_direction)
str_steps = ' '.join(str(s) for s in steps)
A = 'L 10 R 8 R 8'
B = 'L 10 L 12 R 8 R 10'
C = 'R 10 L 12 R 10'
M = str_steps.replace(A, 'A').replace(B, 'B').replace(C, 'C')

In [15]:
def make_ascii_instructions(instructions):
    ascii = []
    instructions = instructions.replace(' ', ',')
    for c in instructions:
        ascii.append(ord(c))
    assert len(ascii) <= 20, ascii
    ascii.append(ord('\n'))
    return ascii

In [16]:
input_values = (
    make_ascii_instructions(M) +
    make_ascii_instructions(A) +
    make_ascii_instructions(B) +
    make_ascii_instructions(C) + 
    make_ascii_instructions('n')
)

In [17]:
print("Part 2:")
print(process_intcode(intcode, input_values))

Part 2:
1016741
